In [19]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset

In [20]:
ratings_df = pd.read_csv('./preprocessed_data/ratings_small_preprocessed.csv', index_col=0)

reader = Reader(rating_scale=(0,10))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)


In [21]:
trainset = data.build_full_trainset()


In [22]:
from surprise.prediction_algorithms.knns import KNNWithMeans

k = 40
min_k = 4
sim_options = {'name': 'cosine', 'user_based': True}

KNNalgo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options)
KNNalgo.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [27]:
KNNalgo.predict(71, 865)

Prediction(uid=71, iid=865, r_ui=None, est=8.75, details={'actual_k': 1, 'was_impossible': False})